# Prérequis

Premièrement, nous avons besoin d'importer les bibliothèques python nécessaires pour notre algorithme.

La bibliothèque la plus importante est py_entitymatching, qui est développée par Anhais Group dans le cadre du projet Magellan.

https://sites.google.com/site/anhaidgroup/home

In [ ]:
# importation des bibliothèques requises (notamment py_entitymatching)

!pip install unidecode
!pip install fuzzywuzzy
!pip install missingno
!pip uninstall Cython -y
!pip uninstall py_entitymatching -y
!pip uninstall py_stringsimjoin -y
!pip install py_stringsimjoin==0.3.3
!pip install py_entitymatching==0.4.1


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
from fuzzywuzzy import fuzz
from unidecode import unidecode
import missingno as msno
import py_stringsimjoin as ssj
import py_stringmatching as sm
import py_entitymatching as em
import math

--------

------------

In [2]:
# ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
# ban_reduced = pd.read_csv('/content/ban_decomposed-12.csv')

# ban_reduced = ban_reduced[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
# nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
# ban_reduced = ban_reduced.rename(columns=nom_colonnes)

# ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
# ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')


#si vous avez enregistré les dataframes de la section précédente sou forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')


--------

# Training

Cet algorithme diffère du premier car il est basé sur du machine learning.
Nous lui fournissons des exemples de comparaisons d'adresses (adresses similaires ou non simialires) contenu dans le fichier dataset_annote_V3.csv, et le modèle apprend par lui même les règles de matching. Il s'agit d'un problème de classification binaire en machine learning.
Ainsi, la qualité des prédictions dépendent de la taille, la qualité et l'exaustivité des exemples fournis dans le dataset annoté.

In [3]:
labeled = pd.read_csv('/content/dataset_annote_V3.csv')
labeled = labeled.dropna(subset=['id','IdentifiantImmeuble'])


labeled = labeled.iloc[:-11,:]

labeled = labeled[~labeled['id'].duplicated()]
labeled = labeled[~labeled['IdentifiantImmeuble'].duplicated()]
labeled = labeled.fillna('')

labeled = labeled.astype({'id': 'string', 'numero' : 'float32' , 'rep' : 'string', 'TypeVoie': 'string', 'NomVoieReste': 'string', 'code_postal': 'int32',
                          'nom_commune': 'string', 'x': 'float32', 'y': 'float32', 'IdentifiantImmeuble': 'string', 'NumeroVoieImmeuble': 'float32',
                          'ComplementNumeroVoieImmeuble': 'string', 'TypeVoieImmeuble': 'string', 'NomVoieImmeuble': 'string',
                          'CodePostalImmeuble': 'int32', 'CommuneImmeuble': 'string', 'CoordonneeImmeubleX': 'float32',
                          'CoordonneeImmeubleY': 'float32', 'Label': 'int32'})

A = labeled.iloc[:, :len(labeled.columns)//2]
B = labeled.iloc[:, len(labeled.columns)//2:-1]
labeled

,id,numero,rep,TypeVoie,NomVoieReste,code_postal,nom_commune,x,y,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY,Label
0,12065_0062_00035,46.0,bis,Avenue,de l'hôte,98120,Pary,122621.109375,1240779.5,IMB/12065/X/0087,46.0,,Rue,de l'hôtel,98120,Pary,122621.109375,1240779.5,0
1,12065_0062_00036,54.0,C,Place,de l'herbier,88120,Castelnau,142621.109375,1440779.5,IMB/12065/X/0088,54.0,C,Place,de l'herbe,88120,Castelnau,142621.109375,1440779.5,0
2,12065_0062_00031,33.0,ter,Route,de Pary,78120,Castres,162621.109375,1640779.5,IMB/12065/X/0083,33.0,T,Route,de Paris,78120,Castres,162521.109375,1640379.5,0
3,12065_0062_00033,6.0,,Impasse,de Saint Thomas,58120,Ile d'Ieux,362621.125000,3640779.5,IMB/12065/X/0085,6.0,,Ruelle,de St-Tomas,58120,Ile d'Ieux,352621.125000,3540779.5,0
4,12140_b267_03121,3121.0,,route,de marroule,12200,Martiel,611103.375000,6366597.5,IMB/12208/X/003V,522.0,,avenue,de Millau,12400,St-Affrique,692138.437500,6316572.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,12063_0466_00014,14.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12057/X/00EK,1.0,,rue,de la Côte Vieille,12120,Cassagnes-Bégonhès,662296.000000,6341233.5,0
953,12063_0466_00012,12.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12102/X/00J5,2.0,,avenue,du Lévézou,12450,Flavin,668608.812500,6354218.0,0
954,12063_0466_00010,10.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12300/X/028I,9.0,,impasse,Sainte-Marguerite,12200,Villefranche-de-Rouergue,623160.562500,6362828.0,0
955,12115_0180_00001,1.0,,route,du Grand Chemin,12230,L'Hospitalet-du-Larzac,715689.812500,6319547.0,IMB/12208/X/01O7,41.0,,route,du Roc Blanc,12400,St-Affrique,690445.937500,6317374.5,0


In [4]:
B['TypeVoieImmeuble'] = B['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')

def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['rep'] = A['rep'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


In [5]:
ban = A.copy()
ipe = B.copy()
labeled = pd.concat([ban,ipe,labeled.iloc[:,-1]],axis=1)
nom_colonnes = {'id': 'ltable_id','numero' : 'ltable_NumeroVoieImmeuble','rep' : 'ltable_ComplementNumeroVoieImmeuble','TypeVoie' : 'ltable_TypeVoieImmeuble','NomVoieReste' : 'ltable_NomVoieImmeuble','code_postal' : 'ltable_CodePostalImmeuble','nom_commune' : 'ltable_CommuneImmeuble','x' : 'ltable_CoordonneeImmeubleX','y' : 'ltable_CoordonneeImmeubleY', 'IdentifiantImmeuble' : 'rtable_id', 'NumeroVoieImmeuble' : 'rtable_NumeroVoieImmeuble', 'ComplementNumeroVoieImmeuble' : 'rtable_ComplementNumeroVoieImmeuble', 'TypeVoieImmeuble' : 'rtable_TypeVoieImmeuble', 'NomVoieImmeuble' : 'rtable_NomVoieImmeuble', 'CodePostalImmeuble' : 'rtable_CodePostalImmeuble', 'CommuneImmeuble': 'rtable_CommuneImmeuble', 'CoordonneeImmeubleX' : 'rtable_CoordonneeImmeubleX', 'CoordonneImmeubleY' : 'rtable_CoordonneeImmeubleY', 'Label' : 'label'}
labeled = labeled.rename(columns = nom_colonnes)

labeled['ltable_TypeVoieImmeuble'] = labeled['ltable_TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')

labeled



,ltable_id,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_id,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,CoordonneeImmeubleY,label
0,12065_0062_00035,46.0,b,avenue,de l'hôte,98120,Pary,122621.109375,1240779.5,IMB/12065/X/0087,46.0,,rue,de l'hôtel,98120,Pary,122621.109375,1240779.5,0
1,12065_0062_00036,54.0,c,place,de l'herbier,88120,Castelnau,142621.109375,1440779.5,IMB/12065/X/0088,54.0,c,place,de l'herbe,88120,Castelnau,142621.109375,1440779.5,0
2,12065_0062_00031,33.0,t,route,de Pary,78120,Castres,162621.109375,1640779.5,IMB/12065/X/0083,33.0,t,route,de Paris,78120,Castres,162521.109375,1640379.5,0
3,12065_0062_00033,6.0,,impasse,de Saint Thomas,58120,Ile d'Ieux,362621.125000,3640779.5,IMB/12065/X/0085,6.0,,ruelle,de St-Tomas,58120,Ile d'Ieux,352621.125000,3540779.5,0
4,12140_b267_03121,3121.0,,route,de marroule,12200,Martiel,611103.375000,6366597.5,IMB/12208/X/003V,522.0,,avenue,de Millau,12400,St-Affrique,692138.437500,6316572.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,12063_0466_00014,14.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12057/X/00EK,1.0,,rue,de la Côte Vieille,12120,Cassagnes-Bégonhès,662296.000000,6341233.5,0
953,12063_0466_00012,12.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12102/X/00J5,2.0,,avenue,du Lévézou,12450,Flavin,668608.812500,6354218.0,0
954,12063_0466_00010,10.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12300/X/028I,9.0,,impasse,Sainte-Marguerite,12200,Villefranche-de-Rouergue,623160.562500,6362828.0,0
955,12115_0180_00001,1.0,,route,du Grand Chemin,12230,L'Hospitalet-du-Larzac,715689.812500,6319547.0,IMB/12208/X/01O7,41.0,,route,du Roc Blanc,12400,St-Affrique,690445.937500,6317374.5,0


In [6]:
nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
A = A.rename(columns=nom_colonnes)

In [7]:
labeled['_id'] = range(len(labeled))

#We need to set key for each table to create metadata so that we can continue the next step
em.set_key(labeled,'_id')
em.set_fk_ltable(labeled, 'ltable_id')
em.set_fk_rtable(labeled, 'rtable_id')
em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')
em.set_ltable(labeled, A)
em.set_rtable(labeled, B)

labeled


,ltable_id,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_id,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,CoordonneeImmeubleY,label,_id
0,12065_0062_00035,46.0,b,avenue,de l'hôte,98120,Pary,122621.109375,1240779.5,IMB/12065/X/0087,46.0,,rue,de l'hôtel,98120,Pary,122621.109375,1240779.5,0,0
1,12065_0062_00036,54.0,c,place,de l'herbier,88120,Castelnau,142621.109375,1440779.5,IMB/12065/X/0088,54.0,c,place,de l'herbe,88120,Castelnau,142621.109375,1440779.5,0,1
2,12065_0062_00031,33.0,t,route,de Pary,78120,Castres,162621.109375,1640779.5,IMB/12065/X/0083,33.0,t,route,de Paris,78120,Castres,162521.109375,1640379.5,0,2
3,12065_0062_00033,6.0,,impasse,de Saint Thomas,58120,Ile d'Ieux,362621.125000,3640779.5,IMB/12065/X/0085,6.0,,ruelle,de St-Tomas,58120,Ile d'Ieux,352621.125000,3540779.5,0,3
4,12140_b267_03121,3121.0,,route,de marroule,12200,Martiel,611103.375000,6366597.5,IMB/12208/X/003V,522.0,,avenue,de Millau,12400,St-Affrique,692138.437500,6316572.5,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,12063_0466_00014,14.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12057/X/00EK,1.0,,rue,de la Côte Vieille,12120,Cassagnes-Bégonhès,662296.000000,6341233.5,0,919
953,12063_0466_00012,12.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12102/X/00J5,2.0,,avenue,du Lévézou,12450,Flavin,668608.812500,6354218.0,0,920
954,12063_0466_00010,10.0,,route,de redoules,12230,La Cavalerie,713384.437500,6324302.0,IMB/12300/X/028I,9.0,,impasse,Sainte-Marguerite,12200,Villefranche-de-Rouergue,623160.562500,6362828.0,0,921
955,12115_0180_00001,1.0,,route,du Grand Chemin,12230,L'Hospitalet-du-Larzac,715689.812500,6319547.0,IMB/12208/X/01O7,41.0,,route,du Roc Blanc,12400,St-Affrique,690445.937500,6317374.5,0,922


In [8]:
G = em.sample_table(labeled, labeled.shape[0])

Cette méthode crée elle même des caractéristiques sur lesquelles le modèle va s'appuyer (par exemple distance Leventshein entre le nom de la commune).

Cependant, certaines features essentielles ne sont pas automatiquement générées, comme la distance entre les coordonnées. Ainsi, nous devons les créer à la main, c'est donc le but de la cellule ci-dessous de rajouter la feature 'real_distance', soit la distance entre les 2 coordonnées, qui sera prise en compte dans le modèle de machine learning.  

In [9]:
#Customize the calculation of distance between 2 tuples
def real_distance(ltuple,rtuple):
  return np.sqrt((ltuple['CoordonneeImmeubleX']-rtuple['CoordonneeImmeubleX'])**2+(ltuple['CoordonneeImmeubleY']-rtuple['CoordonneeImmeubleY'])**2)

match_f = em.get_features_for_matching(A,B, validate_inferred_attr_types = False)

em.add_blackbox_feature(match_f,'real_distance',real_distance)
feature_to_remain = [6,13,23,26,38,54]
match_f.drop([i for i in range(match_f.shape[0]) if i not in feature_to_remain],inplace=True)
match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
6,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,NumeroVoieImmeuble,NumeroVoieImmeuble,None,None,exact_match,<function NumeroVoieImmeuble_NumeroVoieImmeuble_exm at 0x7ae996225ea0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
13,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,ComplementNumeroVoieImmeuble,ComplementNumeroVoieImmeuble,None,None,monge_elkan,<function ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel at 0x7ae996227250>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
23,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,TypeVoieImmeuble,TypeVoieImmeuble,None,None,lev_sim,<function TypeVoieImmeuble_TypeVoieImmeuble_lev_sim at 0x7ae9962277f0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
26,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,NomVoieImmeuble,NomVoieImmeuble,qgm_3,qgm_3,jaccard,<function NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3 at 0x7ae9962279a0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
38,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble,CommuneImmeuble,qgm_3,qgm_3,jaccard,<function CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3 at 0x7ae9950a40d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
54,real_distance,None,None,None,None,None,<function real_distance at 0x7ae996226b90>,None,False


In [10]:
H_train = em.extract_feature_vecs(G, feature_table=match_f , attrs_after=['label'])
H_train

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,label
0,0,12065_0062_00035,IMB/12065/X/0087,1,0.0,0.333333,0.642857,1.000000,0.000000,0
1,1,12065_0062_00036,IMB/12065/X/0088,1,1.0,0.800000,0.529412,1.000000,0.000000,0
2,2,12065_0062_00031,IMB/12065/X/0083,1,1.0,0.800000,0.461538,1.000000,412.310563,0
3,3,12065_0062_00033,IMB/12065/X/0085,1,1.0,0.142857,0.363636,1.000000,100498.756211,0
4,4,12140_b267_03121,IMB/12208/X/003V,0,1.0,0.166667,0.142857,0.000000,95232.252832,0
...,...,...,...,...,...,...,...,...,...,...
952,919,12063_0466_00014,IMB/12057/X/00EK,0,1.0,0.600000,0.100000,0.000000,53821.038065,0
953,920,12063_0466_00012,IMB/12102/X/00J5,0,1.0,0.166667,0.041667,0.000000,53850.010679,0
954,921,12063_0466_00010,IMB/12300/X/028I,0,1.0,0.142857,0.000000,0.025641,98105.047250,0
955,922,12115_0180_00001,IMB/12208/X/01O7,0,1.0,1.000000,0.107143,0.000000,25337.185741,0


In [11]:
H_train = em.impute_table(H_train, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], strategy='mean')

In [12]:
dt = em.RFMatcher(n_estimators=300, max_depth = 15)

In [13]:
train_test = em.split_train_test(H_train, train_proportion=0.7)
devel_set = train_test['train']
eval_set = train_test['test']

In [14]:
dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')


In [15]:
pred_table = dt.predict(table=eval_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id','label'], target_attr='predicted_labels', append=True, inplace=True)

In [16]:
pred_table

,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,label,predicted_labels
759,739,12208_0880_00005,IMB/12208/X/02IW,1.0,1.0,1.000000,1.000000,0.611111,13.366382,1.0,1.0
583,575,12131_0036_00781,IMB/12131/X/000L,1.0,1.0,0.833333,0.600000,1.000000,11.502717,1.0,1.0
82,81,12105_0205_00005,IMB/12105/X/00AP,1.0,1.0,1.000000,0.473684,1.000000,13.038405,1.0,1.0
791,771,12303_0044_00012,IMB/12303/X/001N,1.0,1.0,1.000000,1.000000,1.000000,12.954397,1.0,1.0
880,859,12084_b094_00010,IMB/12084/X/00K9,1.0,1.0,1.000000,1.000000,1.000000,13.164969,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
127,126,12052_0790_00009,IMB/12197/X/00RR,0.0,1.0,0.600000,0.000000,0.000000,69860.717449,0.0,0.0
470,464,12161_jnwe6e_00125,IMB/12161/X/00BS,1.0,1.0,1.000000,1.000000,1.000000,13.101169,1.0,1.0
257,251,12089_0870_00009,IMB/12089/X/01DB,1.0,1.0,0.666667,1.000000,1.000000,9.219544,1.0,1.0
195,191,12013_0033_00005,IMB/12013/X/000R,0.0,1.0,0.833333,0.793103,1.000000,376.343261,0.0,0.0


In [17]:
pred_table[pred_table['label']==1.0]

,_id,ltable_id,rtable_id,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,label,predicted_labels
759,739,12208_0880_00005,IMB/12208/X/02IW,1.0,1.0,1.000000,1.000000,0.611111,13.366382,1.0,1.0
583,575,12131_0036_00781,IMB/12131/X/000L,1.0,1.0,0.833333,0.600000,1.000000,11.502717,1.0,1.0
82,81,12105_0205_00005,IMB/12105/X/00AP,1.0,1.0,1.000000,0.473684,1.000000,13.038405,1.0,1.0
791,771,12303_0044_00012,IMB/12303/X/001N,1.0,1.0,1.000000,1.000000,1.000000,12.954397,1.0,1.0
880,859,12084_b094_00010,IMB/12084/X/00K9,1.0,1.0,1.000000,1.000000,1.000000,13.164969,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
148,147,12030_0012_00002,IMB/12030/X/0006,1.0,1.0,1.000000,1.000000,1.000000,1408.520657,1.0,1.0
12,12,12140_s64usw_00016,IMB/12140/X/009A,1.0,1.0,1.000000,0.600000,1.000000,12.669014,1.0,1.0
632,624,12050_0060_00024,IMB/12050/X/006F,1.0,1.0,1.000000,1.000000,1.000000,13.177277,1.0,1.0
470,464,12161_jnwe6e_00125,IMB/12161/X/00BS,1.0,1.0,1.000000,1.000000,1.000000,13.101169,1.0,1.0


In [18]:
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

OrderedDict([('prec_numerator', 129.0),
             ('prec_denominator', 129.0),
             ('precision', 1.0),
             ('recall_numerator', 129.0),
             ('recall_denominator', 129.0),
             ('recall', 1.0),
             ('f1', 1.0),
             ('pred_pos_num', 129.0),
             ('false_pos_num', 0.0),
             ('false_pos_ls', []),
             ('pred_neg_num', 149.0),
             ('false_neg_num', 0.0),
             ('false_neg_ls', [])])

On observe ici que nous avons une **précision de 91%**, ce qui signifie que 91% des adresses identifiées comme similaires le sont effectivement.

A noter : à chaque réentrainement du modèle, la précision peut légèrement varier.


In [19]:
'''
dt = em.DTMatcher()
rf = em.RFMatcher()
result = em.select_matcher(matchers=[dt, rf], table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label', k=5)
'''

"\ndt = em.DTMatcher()\nrf = em.RFMatcher()\nresult = em.select_matcher(matchers=[dt, rf], table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label', k=5)\n"

# Prediction

Dans cette partie, nous utilisons le modèle entrainé sur le dataset annoté d'entrainement afin de prédire si les adresses sont similaires ou non.

In [20]:
#si vous avez enregistré les dataframes de la section précédente sous forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')

In [21]:
A = ipe_reduced.copy()
B = ban_reduced.copy()

#A.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)
#B.rename(columns={'IdentifiantImmeuble': 'id'}, inplace=True)

em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')


shape_A = A.shape
shape_B = B.shape
print(shape_A)
print(shape_B)

(283831, 9)
(152683, 9)


In [22]:
def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['ComplementNumeroVoieImmeuble'] = A['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


### Blocking

Comme pour l'algorithme 1 basé sur nos propres règles, nous définissons des colonnes pour lesquelles la correspondance doit être stricte entre les 2 adresses :
- NumeroVoieImmeuble
- ComplementNumeroVoieImmeuble
- TypeVoieImmeuble
- CodePostalImmeuble


In [23]:
# define the blocker attributes to check and put in 1 column (because AttrEquivalenceBlocker can only handle 1 column comparison)
# for the unknown NumeroVoieImmeuble, we impute with the value 0
'''
A['NumeroVoieImmeuble'] = A['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(A[A['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the IPE dataset')

B['NumeroVoieImmeuble'] = B['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(B[B['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the BAN dataset')

A['CodePostalImmeuble'] = A['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(A[A['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the IPE dataset')

B['CodePostalImmeuble'] = B['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(B[B['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the BAN dataset')
'''



A['blocking_data'] = A['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  A['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + A['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + A['CodePostalImmeuble'].fillna('').astype(str)
B['blocking_data'] = B['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  B['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + B['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + B['CodePostalImmeuble'].fillna('').astype(str)



In [24]:
attributes = ['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']

ab = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C = ab.block_tables(A, B,
                    l_block_attr='blocking_data', r_block_attr='blocking_data',
                    l_output_attrs=attributes,
                    r_output_attrs=attributes,
                    l_output_prefix='ltable_', r_output_prefix='rtable_')

C

,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,ltable_NumeroVoieImmeuble,ltable_ComplementNumeroVoieImmeuble,ltable_TypeVoieImmeuble,ltable_NomVoieImmeuble,ltable_CodePostalImmeuble,ltable_CommuneImmeuble,ltable_CoordonneeImmeubleX,ltable_CoordonneeImmeubleY,rtable_NumeroVoieImmeuble,rtable_ComplementNumeroVoieImmeuble,rtable_TypeVoieImmeuble,rtable_NomVoieImmeuble,rtable_CodePostalImmeuble,rtable_CommuneImmeuble,rtable_CoordonneeImmeubleX,rtable_CoordonneeImmeubleY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.7426,6344054.694,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.2682,6344332.362,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.5716,6344372.058,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.0282,6347025.643,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43


### ML prediction

In [25]:
H = em.extract_feature_vecs(C, feature_table=match_f)
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], strategy='mean')
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:04:49


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance
0,0,IMB/12001/X/0001,12001_0052_00004,1.0,1.0,1.0,0.000000,1.0,869.844563
1,1,IMB/12001/X/0001,12001_0040_00004,1.0,1.0,1.0,0.028571,1.0,912.011620
2,2,IMB/12001/X/0001,12001_0083_00004,1.0,1.0,1.0,1.000000,1.0,33.955654
3,3,IMB/12001/X/0001,12001_0067_00004,1.0,1.0,1.0,0.250000,1.0,2957.680514
4,4,IMB/12001/X/0001,12001_0069_00004,1.0,1.0,1.0,0.222222,1.0,512.126658
...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,1.0,1.0,1.0,1.000000,1.0,40.512861
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,44.155615
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,47.087184
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,1.0,1.0,1.0,1.000000,1.0,18.890861


In [26]:
print(devel_set.columns)
print(H.columns)

Index(['_id', 'ltable_id', 'rtable_id',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_exm',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel',
       'TypeVoieImmeuble_TypeVoieImmeuble_lev_sim',
       'NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3',
       'CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3', 'real_distance',
       'label'],
      dtype='object')
Index(['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble',
       'NumeroVoieImmeuble_NumeroVoieImmeuble_exm',
       'ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel',
       'TypeVoieImmeuble_TypeVoieImmeuble_lev_sim',
       'NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3',
       'CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3', 'real_distance'],
      dtype='object')


C'est dans cette cellule que nous utilisons notre modèle de DecisionTree  afin d'évaluer si 2 adresses sont similaires ou non.
Si les adresses sont similaires, predicted_labels = 1, et 0 sinon.
Le désavantage de cette méthode est que la prédiciton est binaire, et donc contrairement au premier algorithme proposé, on n'obtient pas réellement un % de confiance de la prédiction, même si nous pouvons le recalculer grâce aux mêmes méthodes que dans le 1er algorithme (Levensthein distance).

En réalité, cet algorithme de machine learning se base aussi sur des calculs de distance entre les différentes composantes de l'adresse, mais il utilise un processus d'arbre de décision afin d'optimiser la sélection.

In [27]:
#dt.fit(table=devel_set, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], target_attr='label')
#pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble','predicted_labels'], target_attr='predicted_labels', append=True, inplace=True)
pred_table = dt.predict(table=H, exclude_attrs=['_id', 'ltable_IdentifiantImmeuble', 'rtable_IdentifiantImmeuble'], target_attr='predicted_labels', append=True, inplace=True)

pred_table


,_id,ltable_IdentifiantImmeuble,rtable_IdentifiantImmeuble,NumeroVoieImmeuble_NumeroVoieImmeuble_exm,ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_mel,TypeVoieImmeuble_TypeVoieImmeuble_lev_sim,NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3,CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3,real_distance,predicted_labels
0,0,IMB/12001/X/0001,12001_0052_00004,1.0,1.0,1.0,0.000000,1.0,869.844563,0.0
1,1,IMB/12001/X/0001,12001_0040_00004,1.0,1.0,1.0,0.028571,1.0,912.011620,0.0
2,2,IMB/12001/X/0001,12001_0083_00004,1.0,1.0,1.0,1.000000,1.0,33.955654,1.0
3,3,IMB/12001/X/0001,12001_0067_00004,1.0,1.0,1.0,0.250000,1.0,2957.680514,0.0
4,4,IMB/12001/X/0001,12001_0069_00004,1.0,1.0,1.0,0.222222,1.0,512.126658,0.0
...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,1.0,1.0,1.0,1.000000,1.0,40.512861,1.0
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,44.155615,1.0
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,1.0,1.0,1.0,1.000000,1.0,47.087184,1.0
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,1.0,1.0,1.0,1.000000,1.0,18.890861,1.0


In [28]:
print('We found', len(pred_table[pred_table['predicted_labels'] == 1]), 'matching adresses')

We found 79259 matching adresses


In [29]:
pred_table[pred_table['predicted_labels'] == 1].to_csv('matched_address.csv')

In [30]:
importances = dt.clf.feature_importances_
# Get indices of features sorted by importance
indices = np.argsort(importances)[::-1]

# Print feature ranking
print("Feature ranking:")
for f in range(6):
    print("%d. feature %str (%f)" % (f + 1, devel_set.columns[3:-1][indices[f]], importances[indices[f]]))

Feature ranking:
1. feature NumeroVoieImmeuble_NumeroVoieImmeuble_exmtr (0.360255)
2. feature real_distancetr (0.249150)
3. feature NomVoieImmeuble_NomVoieImmeuble_jac_qgm_3_qgm_3tr (0.203281)
4. feature CommuneImmeuble_CommuneImmeuble_jac_qgm_3_qgm_3tr (0.119745)
5. feature TypeVoieImmeuble_TypeVoieImmeuble_lev_simtr (0.058229)
6. feature ComplementNumeroVoieImmeuble_ComplementNumeroVoieImmeuble_meltr (0.009340)


In [31]:
match_table = pred_table[pred_table['predicted_labels'] == 1]
# Sort the DataFrame by column 'B' in ascending order
df_sorted = match_table.sort_values(by='real_distance')

# Keep only the first occurrence of each value in column 'A' (which has the minimum value in column 'B')
match_table1 = df_sorted.drop_duplicates(subset='ltable_IdentifiantImmeuble', keep='first')
match_table2 = match_table1.drop_duplicates(subset='rtable_IdentifiantImmeuble', keep='first')
print(len(match_table2))

72883
